# <center> Kafka Producer for Twitter </center>

In [ ]:
!/usr/hdp/current/kafka-broker/bin/kafka-topics.sh --list --zookeeper clnode222.clemson.cloudlab.us:2181

In [ ]:
from confluent_kafka import Producer
import sys

import logging
import json

from datetime import datetime

from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

broker = 'clnode225.clemson.cloudlab.us:6667'
topic = 'twitter'

# Producer configuration
# See https://github.com/edenhill/librdkafka/blob/master/CONFIGURATION.md
conf = {'bootstrap.servers': broker}

# Create Producer instance
p = Producer(**conf)

# Optional per-message delivery callback (triggered by poll() or flush())
# when a message has been successfully delivered or permanently
# failed delivery (after retries).
def delivery_callback(err, msg):
    if err:
        sys.stderr.write('%% Message failed delivery: %s\n' % err)
    else:
        sys.stderr.write('%% Message key %s delivered to %s [%d]\n' % (msg.key(), msg.topic(), msg.partition()))

class StdOutListener(StreamListener):
    def on_data(self, data):
        try:
            jsonData = json.loads(data)
            p.produce(topic, json.dumps(data), key=str(jsonData['id']), callback=delivery_callback)
        except BufferError as e:
            sys.stderr.write('%% Local producer queue is full (%d messages awaiting delivery): try again\n' % len(p))
        p.poll(0)
        return True
    def on_error(self, status):
        print (status)
        
# read cert (not on github)
keyFile = open('/users/lngo/.cert','r')
CONSUMER_KEY = keyFile.readline().rstrip()
CONSUMER_SECRET = keyFile.readline().rstrip()
ACCESS_TOKEN_KEY = keyFile.readline().rstrip()
ACCESS_TOKEN_SECRET = keyFile.readline().rstrip()

auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN_KEY, ACCESS_TOKEN_SECRET)

while True:
    try:
        stream = Stream(auth, StdOutListener())
        stream.filter(track=['Clemson'])
    except IncompleteRead:
        print ("Lets try it again")
        continue
    except KeyboardInterrupt:
        stream.disconnect()
        break
        
# Wait until all messages have been delivered
sys.stderr.write('%% Waiting for %d deliveries\n' % len(p))
p.flush()